In [1]:
!pip3 install pandas
!pip3 install tensorflow
!pip3 install tensorflow-probability
!pip3 install pybullet
!pip3 install jsonpickle

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfpd = tfp.distributions
import pandas as pd
import numpy as np
import pybullet as p
import time
import pybullet_data
import jsonpickle
import os
from grippers import Suction

2022-11-03 00:45:05.911548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 00:45:05.992263: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-03 00:45:06.379717: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-03 00:45:06.379747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
df = pd.read_csv('data_points.txt', sep=" ", header=None)
# df.columns = [c for c in (i+'px',i+'py',i+'pz',i+'dx',i+'dy',i+'dz') for i in range(1,4)]
df.columns = ['1cube','1cuboid','1thin_cuboid','1thinner_cuboid','1px','1py','1pz','1dx','1dy','1dz','2cube','2cuboid','2thin_cuboid','2thinner_cuboid','2px','2py','2pz','2dx','2dy','2dz','3cube','3cuboid','3thin_cuboid','3thinner_cuboid','3px','3py','3pz','3dx','3dy','3dz','NaN']
df.drop(df.columns[-1], axis=1, inplace=True)
pd.set_option('display.max_columns', None)

df.head()

,1cube,1cuboid,1thin_cuboid,1thinner_cuboid,1px,1py,1pz,1dx,1dy,1dz,2cube,2cuboid,2thin_cuboid,2thinner_cuboid,2px,2py,2pz,2dx,2dy,2dz,3cube,3cuboid,3thin_cuboid,3thinner_cuboid,3px,3py,3pz,3dx,3dy,3dz
0,1.0,0.0,0.0,0.0,0.516017,0.005864,0.101467,0.03,0.030,0.03,0.0,0.0,0.0,1.0,0.486154,-0.011845,0.086978,0.03,0.030,0.01,1.0,0.0,0.0,0.0,0.504393,0.016517,0.088899,0.03,0.030,0.03
1,1.0,0.0,0.0,0.0,0.512485,0.000377,0.097769,0.03,0.030,0.03,1.0,0.0,0.0,0.0,0.508266,0.012285,0.081395,0.03,0.030,0.03,1.0,0.0,0.0,0.0,0.502419,0.013539,0.084958,0.03,0.030,0.03
2,0.0,0.0,1.0,0.0,0.487709,-0.016338,0.094075,0.06,0.025,0.01,0.0,0.0,1.0,0.0,0.498382,0.007402,0.089118,0.06,0.025,0.01,0.0,0.0,0.0,1.0,0.483544,-0.002319,0.106194,0.03,0.030,0.01
3,1.0,0.0,0.0,0.0,0.501543,0.015722,0.102428,0.03,0.030,0.03,1.0,0.0,0.0,0.0,0.511461,-0.013617,0.103282,0.03,0.030,0.03,0.0,0.0,1.0,0.0,0.510984,-0.004826,0.106737,0.06,0.025,0.01
4,0.0,0.0,0.0,1.0,0.513928,-0.005884,0.083713,0.03,0.030,0.01,0.0,0.0,0.0,1.0,0.500029,-0.014089,0.119384,0.03,0.030,0.01,0.0,0.0,1.0,0.0,0.508541,-0.012000,0.114687,0.06,0.025,0.01


In [4]:
df[(df['1cube'] == 1) & (df['2cube'] == 1) & (df['3thin_cuboid'] == 1)]

,1cube,1cuboid,1thin_cuboid,1thinner_cuboid,1px,1py,1pz,1dx,1dy,1dz,2cube,2cuboid,2thin_cuboid,2thinner_cuboid,2px,2py,2pz,2dx,2dy,2dz,3cube,3cuboid,3thin_cuboid,3thinner_cuboid,3px,3py,3pz,3dx,3dy,3dz
3,1.0,0.0,0.0,0.0,0.501543,0.015722,0.102428,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.511461,-0.013617,0.103282,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.510984,-0.004826,0.106737,0.06,0.025,0.01
8,1.0,0.0,0.0,0.0,0.496316,0.008046,0.091590,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.490922,-0.019487,0.092987,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.499337,-0.010681,0.116579,0.06,0.025,0.01
28,1.0,0.0,0.0,0.0,0.519460,0.008673,0.109805,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.509335,0.012895,0.100932,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.507828,0.007900,0.108386,0.06,0.025,0.01
237,1.0,0.0,0.0,0.0,0.518427,-0.015535,0.083316,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.480148,-0.010756,0.091677,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.496761,-0.007474,0.100945,0.06,0.025,0.01
306,1.0,0.0,0.0,0.0,0.495533,0.018224,0.114009,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.496236,-0.018075,0.115174,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.486459,-0.006864,0.108140,0.06,0.025,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,1.0,0.0,0.0,0.0,0.481948,0.016685,0.112026,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.513471,-0.006593,0.103219,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.508685,0.012474,0.114000,0.06,0.025,0.01
4326,1.0,0.0,0.0,0.0,0.519925,-0.005678,0.109815,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.493757,-0.012276,0.109580,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.508748,-0.015142,0.092858,0.06,0.025,0.01
4355,1.0,0.0,0.0,0.0,0.518962,0.016762,0.105171,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.489309,-0.019734,0.119133,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.508601,0.001703,0.105090,0.06,0.025,0.01
4361,1.0,0.0,0.0,0.0,0.510126,-0.010022,0.107235,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.503904,-0.003617,0.088609,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.517620,-0.002648,0.093889,0.06,0.025,0.01


In [5]:
# with open('data_setup.txt', 'r') as f:
#     for i, line in enumerate(f.readlines()):
#         print('line ', i, line)
#         if i == 1:
#             print(jsonpickle.decode(line))
#             print('it is now line 1')
#         else:
#             break
l1 = '''
[{"py/reduce": [{"py/function": "numpy.core.multiarray._reconstruct"}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg=="}]}, {"py/tuple": [1, {"py/tuple": [3]}, {"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["O8", false, true]}, {"py/tuple": [3, "|", null, null, null, -1, -1, 63]}]}, false, [{"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U4", false, true]}, {"py/tuple": [3, "<", null, null, null, 16, 4, 8]}]}, {"py/b64": "YwAAAHUAAABiAAAAZQAAAA=="}]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U4", false, true]}, {"py/tuple": [3, "<", null, null, null, 16, 4, 8]}]}, {"py/b64": "YwAAAHUAAABiAAAAZQAAAA=="}]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U11", false, true]}, {"py/tuple": [3, "<", null, null, null, 44, 4, 8]}]}, {"py/b64": "dAAAAGgAAABpAAAAbgAAAF8AAABjAAAAdQAAAGIAAABvAAAAaQAAAGQAAAA="}]}]}]]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray._reconstruct"}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg=="}]}, {"py/tuple": [1, {"py/tuple": [3, 3]}, {"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["f8", false, true]}, {"py/tuple": [3, "<", null, null, null, -1, -1, 0]}]}, false, {"py/b64": "rFlKqKMM4D+dPw8+jRmQP0mCYU68OLo/gHFc4eNd4D+mG1rT/uKLv3mBkkuvcLo/xLXIpfpZ4D8uctKNksRzvy7bllglU7s/"}]}]}]
'''
t1 = jsonpickle.decode(l1)
t1

[array(['cube', 'cube', 'thin_cuboid'], dtype=object),
 array([[ 0.50154288,  0.01572247,  0.10242822],
        [ 0.5114612 , -0.01361655,  0.10328193],
        [ 0.51098378, -0.00482614,  0.10673746]])]

In [ ]:
l1 = '''
[{"py/reduce": [{"py/function": "numpy.core.multiarray._reconstruct"}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg=="}]}, {"py/tuple": [1, {"py/tuple": [3]}, {"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["O8", false, true]}, {"py/tuple": [3, "|", null, null, null, -1, -1, 63]}]}, false, [{"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U4", false, true]}, {"py/tuple": [3, "<", null, null, null, 16, 4, 8]}]}, {"py/b64": "YwAAAHUAAABiAAAAZQAAAA=="}]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U4", false, true]}, {"py/tuple": [3, "<", null, null, null, 16, 4, 8]}]}, {"py/b64": "YwAAAHUAAABiAAAAZQAAAA=="}]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U11", false, true]}, {"py/tuple": [3, "<", null, null, null, 44, 4, 8]}]}, {"py/b64": "dAAAAGgAAABpAAAAbgAAAF8AAABjAAAAdQAAAGIAAABvAAAAaQAAAGQAAAA="}]}]}]]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray._reconstruct"}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg=="}]}, {"py/tuple": [1, {"py/tuple": [3, 3]}, {"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["f8", false, true]}, {"py/tuple": [3, "<", null, null, null, -1, -1, 0]}]}, false, {"py/b64": "rFlKqKMM4D+dPw8+jRmQP0mCYU68OLo/gHFc4eNd4D+mG1rT/uKLv3mBkkuvcLo/xLXIpfpZ4D8uctKNksRzvy7bllglU7s/"}]}]}]
'''
t1 = jsonpickle.decode(l1)

######

ASSETS_ROOT = os.path.abspath("urdf")
obj_ids = {'fixed': [], 'rigid': [], 'deformable': []}
homej = np.array([-1, -0.5, 0.5, -0.5, -0.5, 0]) * np.pi

physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.81)

target = (-0.07796166092157364, 0.005451506469398737, -0.06238798052072525)
dist = 1.0
yaw = 89.6000747680664
pitch = -17.800016403198242
p.resetDebugVisualizerCamera(dist,yaw,pitch,target)

planeId = p.loadURDF(os.path.join(ASSETS_ROOT, 'plane/plane.urdf'))

origin = np.zeros(3)
noRotation = p.getQuaternionFromEuler([0,0,0])

ur5 = p.loadURDF(os.path.join(ASSETS_ROOT, 'ur5/ur5.urdf'), origin, noRotation, 
                useFixedBase=True)
suc = Suction(ASSETS_ROOT, ur5, 9, obj_ids)
n_joints = p.getNumJoints(ur5)
joints = [p.getJointInfo(ur5, i) for i in range(n_joints)]
joints = [j[0] for j in joints if j[2] == p.JOINT_REVOLUTE]

for j, hj in zip(joints, homej):
    p.resetJointState(ur5, j, hj)

# only static dimension shapes
shapes = ['cube', 'cuboid', 'thin_cuboid', 'thinner_cuboid']
valid_rots = {
    'cube': ([0], [0], [0]),
    'cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
    'thin_cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
    'thinner_cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
}

dims = {
    'cube': np.array([0.03, 0.03, 0.03]),
    'cuboid': np.array([0.03, 0.025, 0.06]),
    'thin_cuboid': np.array([0.06, 0.025, 0.01]),
    'thinner_cuboid': np.array([0.03, 0.03, 0.01])
}

shape_oh = {
    'cube': [1, 0, 0, 0],
    'cuboid': [0, 1, 0, 0],
    'thin_cuboid': [0, 0, 1, 0],
    'thinner_cuboid': [0, 0, 0, 1],
}

obj_num = 3

objPos = np.array([0.5,0,0.1])

# t1[0][0] = 'cube'
t1[0][1] = 'c3'
# t1[1][1] += np.array([0,0,0]) 
t1[0][2] = 'thinner_cuboid'

for i in range(3):
    add = np.array([0,0,0])
    ot = t1[0][i]
    if i == 2:
        add = np.array([+0.003,+0.007,+0.006])
    o1 = p.loadURDF(os.path.join(ASSETS_ROOT, f'objects/{ot}.urdf'), t1[1][i]+add, noRotation)
    for i in range (150):
        p.stepSimulation()
        time.sleep(1./240.)

input()

p.disconnect()

In [7]:
t1

[array(['cube', 'c3', 'thinner_cuboid'], dtype=object),
 array([[ 0.50154288,  0.01572247,  0.10242822],
        [ 0.5114612 , -0.01361655,  0.10328193],
        [ 0.51098378, -0.00482614,  0.10673746]])]

In [8]:
import tensorflow as tf
import tensorflow_probability as tfp
tfpd = tfp.distributions

d = tfpd.JointDistributionSequential([
    tfpd.Uniform(),
    tfpd.Uniform(),
    tfpd.Uniform()
])

print(d.resolve_graph())

(('x2', ()), ('x1', ()), ('x', ()))


2022-11-03 00:45:30.609567: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 00:45:30.633557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 00:45:30.633728: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 00:45:30.634197: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [9]:
p.disconnect()

error: Not connected to physics server.